In [1]:
# load the base and fine-tuned model
import argparse
import multiprocessing
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import transformers
from accelerate import PartialState
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    logging,
    set_seed,
)
#from trl import SFTTrainer
#from peft import PeftModel, PeftConfig
# verify the load

#merged_model_path= f"/usr/project/llm-compressor/starcoder2_7b_22k_ft_80EM-quantized.w8a8_optimal"

merged_model_path = 'bigcode/starcoder2-7b'


quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True)

model = AutoModelForCausalLM.from_pretrained(
    merged_model_path,
    device_map="cuda:0",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

def generate_completion(text: str):
    inputs = tokenizer.encode(text, return_tensors="pt").to("cuda:0")
    #print(len(inputs[0]))
    outputs = model.generate(inputs, max_length=128, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0])



print ('--------------Hyphen Tag-----------------')
text = "<fim-prefix>def fib(n):<fim-suffix>    else:\n        return fib(n - 2) + fib(n - 1)<fim-middle>"
print(generate_completion(text))

print ('--------------Under Score Tag-----------------')
text_underscore= "<fim_prefix>def fib(n):<fim_suffix>    else:\n        return fib(n - 2) + fib(n - 1)<fim_middle>"
print(generate_completion(text_underscore))

/home/syu2/anaconda3/envs/refact3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:15<00:00,  5.01s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--------------Hypthen Tag-----------------
<fim-prefix>def fib(n):<fim-suffix>    else:
        return fib(n - 2) + fib(n - 1)<fim-middle>

<fim-prefix>def fib(n):<fim-suffix>    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 2) + fib(n - 1)<fim-middle>

<fim-prefix>def fib(n):<fim-suffix>   
--------------Under Score Tag-----------------
<fim_prefix>def fib(n):<fim_suffix>    else:
        return fib(n - 2) + fib(n - 1)<fim_middle>
    if n == 0:
        return 0
    elif n == 1:
        return 1
<file_sep><fim_prefix><fim_suffix> 1000000007

def main():
    n = int(input())
    print(fib(n))

if __name__ == '__main__':
    main()<fim_middle>/fibonacci.py
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
       
